In [34]:
import torch
import torch.nn as nn   #all different model (CNN, MLP, ...) 
import torch.optim as optim   #optimization  (SGD)
print(torch.__version__)
import torch.nn.functional as func #like ReLU and tanh
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plot

2.5.1+cpu


In [35]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes): #(28x28 = 784 pixels)
        super(NN, self).__init__()   #ensure parents class being initialize correctly
        self.hlayer1 = nn.Linear(input_size, 50)
        self.hlayer2 = nn.Linear(50,num_classes) #784 - 50 - 10
    def forward(self, x):
        x = func.relu(self.hlayer1(x))
        x = self.hlayer2(x)
        return x


In [36]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [37]:
input_size = 784
num_classes = 10
batch_size = 64
learning_rate = 0.001
num_epochs = 10

In [ ]:
transform = transforms.Compose([         #follow normal default transform
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_datasets = datasets.MNIST(root="./MNIST", train=True,transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_datasets, batch_size= batch_size, shuffle=True)

test_datasets = datasets.MNIST(root="./MNIST", train=False,transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_datasets, batch_size= batch_size, shuffle=True)


In [ ]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        #Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        #bcos we get 28*28 pixel in matrix form to get to 728 vector point we reshape it
        data = data.reshape(data.shape[0], -1)

        #forward
        scores = model(data)  #wow only that? too easy
        loss = criterion(scores, targets)

        #backward
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()  #put in Adam optimizer


In [ ]:
def check_accuracy(loader,model):
    num_correct = 0
    num_samples = 0
    model.eval()   #evaluation mode

    with torch.no_grad():    
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)    #turn x to 2D tensor

            scores = model(x)
            _, predictions = scores.max(1)   #max(1) finds the highest value and predictions to store the indices of the value (like from 0-9 then just need the highest value to find the number, the value itself is not needed)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
        model.train()
        
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)
    

Got 58977 / 60000 with accuracy 98.30
Got 9681 / 10000 with accuracy 96.81
